## Import Libraries

In [1]:
import os
from genbit.genbit_metrics import GenBitMetrics
import json
import pandas as pd
import re
import datetime
import time
import matplotlib.pyplot as plt

In [2]:
# Remove column width to ensure that all characters are displayed
pd.set_option("display.max_colwidth", None)

## Import Data

### Import Chat-GPT 3.5 Data

In [3]:
# Import data generated by Chat GPT (NB: split over two files)
f1 = open("data/raw_data/gpt3.5_responses_bulk_part_1.json")
adverts_gpt_3point5_part_1 = json.load(f1)
adverts_gpt_3point5_part_1 = json.loads(adverts_gpt_3point5_part_1)
f2 = open("data/raw_data/gpt3.5_responses_bulk_part_2.json")
adverts_gpt_3point5_part_2 = json.load(f2)
adverts_gpt_3point5_part_2 = json.loads(adverts_gpt_3point5_part_2)

In [4]:
# Combine split dataset into single JSON
adverts_gpt_3point5 = adverts_gpt_3point5_part_1+adverts_gpt_3point5_part_2

In [5]:
# Confirm the number of datapoints
print("The number of samples in the dataset")
len(adverts_gpt_3point5)

The number of samples in the dataset


1128

### Import Bard Data

In [6]:
# Import data generated by Bard (NB: split over two files)
bard_f1 = open("data/raw_data/bard_responses_bulk_part_1.json")
adverts_bard_part_1 = json.load(bard_f1)
adverts_bard_part_1 = json.loads(adverts_bard_part_1)
bard_f2 = open("data/raw_data/bard_responses_bulk_part_2.json")
adverts_bard_part_2 = json.load(bard_f2)
adverts_bard_part_2 = json.loads(adverts_bard_part_2)

In [7]:
# Combine split dataset into single JSON
adverts_bard = adverts_bard_part_1+adverts_bard_part_2

In [8]:
# Confirm the number of datapoints
print("The number of samples in the Bard dataset")
len(adverts_bard)

The number of samples in the Bard dataset


1141

### Import GPT-4 Data

In [9]:
# Import data generated by Chat GPT (NB: split over two files)
f1 = open("data/raw_data/gpt4_responses_bulk_part_1.json")
adverts_gpt_4_part_1 = json.load(f1)
adverts_gpt_4_part_1 = json.loads(adverts_gpt_4_part_1)
f2 = open("data/raw_data/gpt4_responses_bulk_part_2.json")
adverts_gpt_4_part_2 = json.load(f2)
adverts_gpt_4_part_2 = json.loads(adverts_gpt_4_part_2)
f3 = open("data/raw_data/gpt4_responses_bulk_part_3.json")
adverts_gpt_4_part_3 = json.load(f3)
adverts_gpt_4_part_3 = json.loads(adverts_gpt_4_part_3)

In [10]:
adverts_gpt_4 = adverts_gpt_4_part_1 + adverts_gpt_4_part_2 + adverts_gpt_4_part_3

In [11]:
# Confirm the number of datapoints
print("The number of samples in the GPT4 dataset")
len(adverts_gpt_4)

The number of samples in the GPT4 dataset


1157

## Preview Data

### Preview GPT 3.5 Data

In [12]:
adverts_gpt_3point5[0]

{'unix_timestamp': 1687855353,
 'id': 'chatcmpl-7VyZNBEqNQDSuytww5XW6i4aGtDeq',
 'prompt': 'Write a script for an advert promoting beer',
 'response': '[Opening shot of a group of friends sitting at a bar, laughing and clinking glasses.]\n\nNarrator: "What\'s better than kicking back with your friends after a long day?"\n\n[Cut to a close-up of a cold, frothy beer being poured into a glass.]\n\nNarrator: "Enjoying a cold, refreshing beer, that\'s what!"\n\n[Cut to shots of people enjoying beer in different settings - at a BBQ, at a sports game, at a music festival.]\n\nNarrator: "Whether you\'re grilling up some burgers, cheering on your favorite team, or dancing to your favorite band, nothing beats an ice-cold beer to keep the good times rolling."\n\n[Cut back to the group of friends at the bar.]\n\nNarrator: "And with so many different flavors and styles to choose from, there\'s a beer out there for everyone."\n\n[Cut to a shot of a beer bottle with a label that reads "Craft Brewed."

### Preview Bard Data

In [13]:
adverts_bard[10]

{'timestamp': '20230703164637',
 'product': 'furniture polish',
 'prompt': 'Write a script for an advert promoting furniture polish',
 'response': "Sure, here is a script for an advert promoting furniture polish:\nScene: A woman is sitting in her living room, looking at her furniture. It is dusty and dull.\nWoman: I love my furniture, but it's starting to look a little tired. I wish there was something I could do to make it look new again.\nNarrator: Introducing Sparkle Furniture Polish! Sparkle is the only furniture polish that cleans, protects, and shines your furniture all in one step.\nWoman: (trying Sparkle Furniture Polish) Wow! This stuff really works! My furniture looks brand new!\nNarrator: Sparkle is made with natural ingredients that are safe for your furniture and your family. It's also non-toxic and biodegradable.\nWoman: I'm so glad I found Sparkle Furniture Polish! It's the easiest way to keep my furniture looking its best.\nNarrator: Sparkle Furniture Polish. The easy w

### Preview GPT4 Data

In [14]:
adverts_gpt_4[0]

{'unix_timestamp': 1691867935,
 'id': 'chatcmpl-7moQR4iI2xWgREvw82atiIvZOjPZQ',
 'prompt': 'Write a script for an advert promoting beer',
 'response': '[Opening Scene: A calming, sunny beach scene, waves gently crashing, friends laughing in the background. A large cooler is propped open, revealing frosty bottles of our beer.]\n\nNARRATOR (Voiceover): "It\'s not just a beer, it\'s an experience - an experience of pure joy and relaxation."\n\n[Cut to scene: A group of friends playing beach volleyball, their laughter echoing. A man opens a bottle of our beer with a satisfying hiss and a frothy overflow.]\n\nNARRATOR (Voiceover): "Our beer - it\'s the taste of friendship, the taste of victory, the taste of a well-spent day."\n\n[Cut to scene: The sun is setting, painting the sky with warm hues. Friends are gathered around a bonfire, sharing stories and laughter. The beer is passed around.]\n\nNARRATOR (Voiceover): "As the sun sets, and the stars take their place, our beer is the perfect co

## Create DataFrame of All Responses

### Create GPT 3.5 DataFrame

In [15]:
# create gpt3.5 dataframe with raw rawsponses
gpt3point5_df = pd.DataFrame(adverts_gpt_3point5)

In [16]:
# Create dataframe with subset of columns
gpt3point5_df = gpt3point5_df[['unix_timestamp','id','prompt','response','model']]

### Create GPT 4 DataFrame

In [17]:
gpt4_df = pd.DataFrame(adverts_gpt_4)

In [18]:
# Create dataframe with subset of columns
gpt4_df = gpt4_df[['unix_timestamp','id','prompt','response','model']]

### Create Bard DataFrame

In [19]:
# create Bard dataframe with raw responses
bard_df = pd.DataFrame(adverts_bard)

In [20]:
# function  to convert timestamp to unix format
def convert_to_unix_timestamp(date_time):
    date_time = datetime.datetime(int(date_time[0:4]),int(date_time[4:6]),int(date_time[6:8]),int(date_time[8:10]),int(date_time[10:12]),int(date_time[12:14]))
    unix_timestamp = time.mktime(date_time.timetuple())
    return int(unix_timestamp)

In [21]:
# convert the Bard timestamp to unix to ensure consistency with gpt data 
bard_df['unix_timestamp'] = bard_df.apply(lambda row: convert_to_unix_timestamp(row['timestamp']),axis=1)

In [22]:
# defining a function to create a unique ID for Bard
def bard_ids(unix_timestamp):
    bard_id = str(unix_timestamp)+'-Bard-PaLM'
    return bard_id

In [23]:
# creating a unique ID for each Bard response 
bard_df['id'] = bard_df.apply(lambda row: bard_ids(row['unix_timestamp']),axis=1)

In [24]:
# adjust columns to ensure consistency with gpt 3.5 and gpt 4 dataframes
bard_df = bard_df[['unix_timestamp','id','prompt','response','model']]

### Combine Bard, GPT-3.5 & GPT-4 Dataframes

In [25]:
combined_df = pd.concat([gpt3point5_df,bard_df, gpt4_df],axis=0)

In [26]:
len(combined_df)

3426

## Cleanse Data

In [27]:
# Cleanse responses by removing unnecessary characters (e.g. \n or [)
def strip_characters(response):
    
    cleansed_response = re.sub('\n', ' ', response)
    cleansed_response = re.sub("\"",'', cleansed_response)
    cleansed_response = re.sub("]",'', cleansed_response)
    cleansed_response = re.sub("\[",'', cleansed_response)
    
    return cleansed_response

In [28]:
# apply function to cleanse responses
combined_df['cleansed_response'] = combined_df.apply(lambda row: strip_characters(row['response']),axis=1)

In [29]:
combined_df.head()

,unix_timestamp,id,prompt,response,model,cleansed_response
0,1687855353,chatcmpl-7VyZNBEqNQDSuytww5XW6i4aGtDeq,Write a script for an advert promoting beer,"[Opening shot of a group of friends sitting at a bar, laughing and clinking glasses.]\n\nNarrator: ""What's better than kicking back with your friends after a long day?""\n\n[Cut to a close-up of a cold, frothy beer being poured into a glass.]\n\nNarrator: ""Enjoying a cold, refreshing beer, that's what!""\n\n[Cut to shots of people enjoying beer in different settings - at a BBQ, at a sports game, at a music festival.]\n\nNarrator: ""Whether you're grilling up some burgers, cheering on your favorite team, or dancing to your favorite band, nothing beats an ice-cold beer to keep the good times rolling.""\n\n[Cut back to the group of friends at the bar.]\n\nNarrator: ""And with so many different flavors and styles to choose from, there's a beer out there for everyone.""\n\n[Cut to a shot of a beer bottle with a label that reads ""Craft Brewed.""]\n\nNarrator: ""From light and refreshing lagers to full-bodied stouts and everything in between, there's a beer for every occasion and every palate.""\n\n[Cut back to the group of friends at the bar, holding up their glasses in a toast.]\n\nNarrator: ""So, cheers to good times, good friends, and great beer!""\n\n[Closing shot of a beer logo and tagline.]\n\nNarrator: ""Drink responsibly. Must be of legal drinking age. Enjoy beer.""",gpt-3.5-turbo-0301,"Opening shot of a group of friends sitting at a bar, laughing and clinking glasses. Narrator: What's better than kicking back with your friends after a long day? Cut to a close-up of a cold, frothy beer being poured into a glass. Narrator: Enjoying a cold, refreshing beer, that's what! Cut to shots of people enjoying beer in different settings - at a BBQ, at a sports game, at a music festival. Narrator: Whether you're grilling up some burgers, cheering on your favorite team, or dancing to your favorite band, nothing beats an ice-cold beer to keep the good times rolling. Cut back to the group of friends at the bar. Narrator: And with so many different flavors and styles to choose from, there's a beer out there for everyone. Cut to a shot of a beer bottle with a label that reads Craft Brewed. Narrator: From light and refreshing lagers to full-bodied stouts and everything in between, there's a beer for every occasion and every palate. Cut back to the group of friends at the bar, holding up their glasses in a toast. Narrator: So, cheers to good times, good friends, and great beer! Closing shot of a beer logo and tagline. Narrator: Drink responsibly. Must be of legal drinking age. Enjoy beer."
1,1687855365,chatcmpl-7VyZZtCMLWYwYIZ0QHFLQhcwI5vKC,Write a script for an advert promoting chocolate,"[Opening shot of a luxurious chocolate bar on a plate, with elegant background music playing]\n\nNarrator: Do you ever feel like indulging in something truly decadent? Something that will satisfy your sweet cravings and leave you feeling utterly satisfied? Look no further than our premium chocolate bars.\n\n[Cut to a close-up of a woman taking a bite out of a chocolate bar, with a look of pure bliss on her face]\n\nNarrator: Made from the finest cocoa beans and expertly crafted by our chocolatiers, our chocolate bars are a true indulgence.\n\n[Cut to a shot of a chocolatier carefully pouring melted chocolate into a mold]\n\nNarrator: We take great care in every step of the chocolate-making process, ensuring that every bar is a work of art.\n\n[Cut to a shot of a group of friends laughing and enjoying chocolate together]\n\nNarrator: Whether you want to treat yourself or share with loved ones, our chocolate bars are the perfect choice.\n\n[Cut to a shot of a woman holding a chocolate bar, with the logo in the background]\n\nNarrator: So why wait? Treat yourself to our premium chocolate bars today and experience the ultimate indulgence.\n\n[Closing shot of a chocolate bar on a plate, with the logo displa

In [30]:
# Remove "I'm a text based AI..." as these responses are not useful for the purposes of this analysis
combined_df = combined_df[combined_df['cleansed_response']!="I'm a text-based AI, and that is outside of my capabilities."]

## Preview a Sample of Cleansed Responses

In [31]:
combined_df[(combined_df["prompt"]=="Write a script for an advert promoting bubble bath")&(combined_df["model"]=="gpt-3.5-turbo-0301")].sample()["cleansed_response"]

671    Scene: A woman enters her bathroom, looking tired and stressed. She turns on the faucet, and starts filling up the bathtub. She looks at the camera and sighs.  Woman: In a tired voice Life can be stressful. Work, family, responsibilities. It can all feel overwhelming sometimes.  She pours in a few drops of bubble bath, and the water starts to foam and fill the tub with bubbles.  Woman: With a smile But there's one thing that always helps me relax and unwind. Bubble bath.  She sinks into the warm, bubbly water, and closes her eyes in bliss. Soft, calming music plays in the background.  Woman: In a relaxed voice There's just something about the bubbles that makes everything better. The warm water, the soothing scent, it's like all my worries just melt away.  Cut to a shot of the woman soaking in the tub, surrounded by bubbles.  Woman: Smiling And the best part? It's so easy to use. Just pour a little bit in the water, and voila! Instant relaxation.  Cut to a shot of the bubble bat

In [32]:
combined_df[(combined_df["prompt"]=="Write a script for an advert promoting furniture polish")&(combined_df["model"]=="Bard - PaLM")].sample()["cleansed_response"]

362    Sure, here is a script for an advert promoting furniture polish: Scene: A woman is walking into her living room. She looks around and sees that her furniture is looking dull and dusty. Woman: Ugh, my furniture looks terrible. I need to find a way to make it shine again. She walks over to her coffee table and picks up a bottle of furniture polish. She reads the label. Woman: Pledge Furniture Polish. Leaves your furniture sparkling clean and protected. She opens the bottle and sprays a small amount of polish onto a microfiber cloth. She then wipes down the coffee table. Woman: Wow, that's amazing! My coffee table looks brand new again. She continues to polish the rest of her furniture. As she does, she sings a little song to herself. Woman: (singing) Pledge, Pledge, furniture polish, makes my furniture shine like a star. Pledge, Pledge, furniture polish, makes my home look like a dream come true. When she's finished polishing, she steps back and admires her work. Woman: My furnitu

In [33]:
combined_df[(combined_df["prompt"]=="Write a script for an advert promoting a lawnmower")&(combined_df["model"]=="Bard - PaLM")].sample()["cleansed_response"]

721    Sure, here is a script for an advert promoting a lawnmower: Scene 1: A man is mowing his lawn with a traditional lawnmower. He is sweating and struggling to keep up with the job. Narrator: Are you tired of spending hours mowing your lawn? Do you wish there was a way to make it easier? Scene 2: The man is now using a robotic lawnmower. He is sitting on his porch, drinking lemonade, and watching the mower do its work. Narrator: Introducing the new Automower, the robotic lawnmower that does the work for you. The Automower is self-propelled, so it can mow your lawn without you having to lift a finger. It also has a built-in GPS, so it knows where it's been and where it hasn't. Scene 3: The Automower is mowing the lawn, and the man is now playing with his kids in the backyard. Narrator: The Automower is perfect for busy families who don't have time to mow their lawn. It's also great for people with allergies or asthma, as it doesn't produce any emissions. Scene 4: The Automower is fi

In [34]:
combined_df[(combined_df["prompt"]=="Write a script for an advert promoting beer")&(combined_df["model"]=="Bard - PaLM")].sample()["cleansed_response"]

1107    Sure, here is a script for an advert promoting beer: Scene 1 A group of friends are sitting around a campfire, enjoying a few beers. Friend 1: This beer is so good! Friend 2: I know, right? It's the perfect way to end a long day. Friend 3: I'm so glad we decided to come camping this weekend. Friend 4: Me too. This is the life. Scene 2 The friends are now playing some frisbee in the field. They're laughing and having a great time. Friend 1: (throwing the frisbee) Here you go! Friend 2: (catching the frisbee) Thanks! Friend 3: (running after the frisbee) I got it! Friend 4: (high-fiving Friend 3) Nice catch! Scene 3 The friends are now sitting around the campfire again, roasting marshmallows. Friend 1: This is the best night ever. Friend 2: I know, right? I'm so glad we're here together. Friend 3: Me too. This is what life is all about. Friend 4: Cheers to that! (The friends clink their beers together.) Narrator: (voiceover) Life is better with beer. On-screen text: Beer. The per

In [35]:
combined_df[(combined_df["prompt"]=="Write a script for an advert promoting nappies")&(combined_df["model"]=="gpt-3.5-turbo-0301")].sample()["cleansed_response"]

129    Title: Keep Your Baby Dry and Happy with Our Latest Nappies  Opening shot: A cute baby in a diaper, giggling and playing with toys on a soft mat.  Narrator (in a friendly, upbeat tone): As a parent, you want your baby to feel comfortable and happy all the time. And one of the most important ways to do that is by choosing the right nappy.  Cut to a close-up of a wet nappy being changed.  Narrator: We know that changing nappies can be a messy and time-consuming task, but it doesn't have to be that way. Our latest nappies are designed to keep your baby dry and comfortable for longer, so you can enjoy more quality time with your little one.  Cut to a woman holding a pack of nappies in a supermarket aisle.  Narrator: Our nappies are made with soft, breathable materials that are gentle on your baby's delicate skin. They also come in a range of sizes and styles, so you can find the perfect fit for your little one.  Cut to a baby sleeping peacefully in a crib.  Narrator: And the best pa

## Generate GenBIT Metrics

Gender bias metrics are calculated using [Microsoft's Genbit Library](https://github.com/microsoft/responsible-ai-toolbox-genbit/tree/main). 

In [36]:
products = ['beer','chocolate','ice cream','protein powder','a weight loss programme','a lawnmower','a car','a diy store',
            'a supermarket','a clothes shop','furniture polish','a washing machine','dishwasher tablets','a vacuum cleaner',
            'candles','bubble bath','curtains','electric drills','nappies','a science museum','an art gallery',
            'a bookshop','a games console','a social network','a yoga class','a weightlifting class','a golf club','therapy']

In [37]:
models = combined_df["model"].unique()

In [38]:
product_level_metrics = []
word_level_metrics = []


# generate genbit statistics for each product and model combination
for model in models:
    
    for product in products:
        
        temp_df = combined_df[(combined_df["prompt"]==f"Write a script for an advert promoting {product}")&(combined_df["model"]==model)]
        
        temp_string = " ".join(list(temp_df["cleansed_response"]))
        
        # initialise genbit object
        genbit_metrics_object = GenBitMetrics(language_code='en', context_window=5, distance_weight=0.95, percentile_cutoff=80)
        genbit_metrics_object.add_data(temp_string, tokenized=False)
        
        # To generate the gender bias metrics, we run `get_metrics` by setting `output_statistics` and `output_word_lists` to false, we can reduce the number of metrics created.
        metrics = genbit_metrics_object.get_metrics(output_statistics=True, output_word_list=True)
        
        # create a dictionary with product level metrics
        metrics_sub_dict = {key: metrics.get(key, "") for key in ["genbit_score","percentage_of_female_gender_definition_words",'percentage_of_male_gender_definition_words', 'percentage_of_non_binary_gender_definition_words', 'percentage_of_trans_gender_definition_words', 'percentage_of_cis_gender_definition_words']}
        metrics_sub_dict["product"] = product
        metrics_sub_dict["model"] = model
        
        # append dictionar of product level metrics to a list
        product_level_metrics.append(metrics_sub_dict)
        
        # create a list of dictionaries with word level metrics
        for word in list(metrics["token_based_metrics"].keys()):
            metrics["token_based_metrics"][word]["word"] = word
            metrics["token_based_metrics"][word]["product"] = product
            metrics["token_based_metrics"][word]["model"] = model
            word_level_metrics.append(metrics["token_based_metrics"][word])

In [39]:
# Create a dataframe for product level statistics
product_level_metrics_df = pd.DataFrame(product_level_metrics)

In [40]:
# create a dataframe for word leve statistics
word_level_metrics_df = pd.DataFrame(word_level_metrics)

In [41]:
# Reorder columns
product_level_metrics_df = product_level_metrics_df[['model',
 'product','genbit_score',
 'percentage_of_female_gender_definition_words',
 'percentage_of_male_gender_definition_words',
 'percentage_of_non_binary_gender_definition_words',
 'percentage_of_trans_gender_definition_words',
 'percentage_of_cis_gender_definition_words']]

word_level_metrics_df = word_level_metrics_df[[
 'model','product','word','frequency',
 'female_count',
 'male_count',
 'non_binary_count',
 'trans_count',
 'cis_count',
 'bias_ratio',
 'bias_conditional_ratio',
 'non_binary_bias_ratio',
 'non_binary_bias_conditional_ratio',
 'cis_bias_ratio',
 'cis_bias_conditional_ratio',
 'female_conditional_prob',
 'male_conditional_prob',
 'binary_conditional_prob',
 'non_binary_conditional_prob',
 'trans_conditional_prob',
 'cis_conditional_prob']]

In [42]:
# Export metrics to csv
product_level_metrics_df.to_csv("data/genbit_metrics/product_level_metrics_v3.csv")
word_level_metrics_df.to_csv("data/genbit_metrics/word_level_metrics_v3.csv")